In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, rand, col
from pyspark.sql.types import StringType, IntegerType
import random
import pandas as pd
import pyspark
from datetime import datetime
from pyspark import SparkConf, SparkContext
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler

# SparkConfを作成し、メモリ設定を行う
conf = SparkConf() \
    .setAppName("MyApp") \
    .set("spark.driver.memory", "8g") \
    .set("spark.executor.memory", "8g")

# SparkContextを作成
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MyApp, master=local[*]) created by __init__ at /var/folders/w_/hg905l611296df772h1275kr0000gn/T/ipykernel_3538/2170893724.py:18 

In [21]:
# ランダムデータ生成

def random_value1():
    return random.randint(0, 100)

def random_value2():
    return random.randint(0, 100)

# UDF（ユーザー定義関数）の登録
random_value1_udf = udf(random_value1, IntegerType())
random_value2_udf = udf(random_value2, IntegerType())

# 空のDataFrameを作成
df = spark.range(10000000).withColumn("id", col("id"))

# 各カラムをランダム値で埋める
df = df.withColumn("value1", random_value1_udf())
df = df.withColumn("value2", random_value2_udf())

# "id"カラムを削除して最終DataFrameにする
df = df.drop("id")

assembler = VectorAssembler(inputCols=["value1", "value2"], outputCol="features")

# VectorAssemblerを使って新しいデータフレームを変換
df = assembler.transform(df)

df = df.select("features")

# 結果を表示
df.show(10)

+-----------+
|   features|
+-----------+
|[37.0,33.0]|
|[38.0,57.0]|
|[40.0,17.0]|
|[95.0,87.0]|
|[21.0,37.0]|
|[52.0,79.0]|
| [1.0,77.0]|
|[42.0,24.0]|
|[75.0,89.0]|
|[52.0,16.0]|
+-----------+
only showing top 10 rows



24/07/16 02:30:46 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 9 (TID 37): Attempting to kill Python Worker


In [22]:
type(df)

pyspark.sql.dataframe.DataFrame

In [27]:
df_sample = df.sample(fraction=0.001, seed=1).cache()

24/07/16 02:32:40 WARN CacheManager: Asked to cache already cached data.


In [42]:
# df_sample = transformed_data.sample(fraction=0.001, seed=1).cache()

# Pandas Corr

In [31]:
pdf_sample = df_sample.toPandas()
pdf_sample[['value1', 'value2']] = pd.DataFrame(pdf_sample['features'].tolist(), index=pdf_sample.index)
pdf_sample.corr()

/var/folders/w_/hg905l611296df772h1275kr0000gn/T/ipykernel_3538/1893691228.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  pdf_sample.corr()


,value1,value2
value1,1.000000,0.011023
value2,0.011023,1.000000


In [41]:
# pdf_sample = df_sample.toPandas()
# pdf_sample[feature_attrs_flat] = pd.DataFrame(pdf_sample['features-dep'].tolist(), index=pdf_sample.index)
# pdf_sample.corr()

# PySpark Corr

In [32]:
from pyspark.ml.stat import Correlation

In [38]:
Correlation.corr(df_sample, "features").head()[0]

DenseMatrix(2, 2, [1.0, 0.011, 0.011, 1.0], False)